# Complex Correlated Gaussian Testing

We need to work out all matrix operations with complex data type

In [2]:
import torch as th

import time

import numpy as np
import matplotlib.pyplot as plt  
import seaborn as sns  
%matplotlib inline
sns.set()

In [42]:
#dtype = th.float64
dtype = th.cdouble

gpuid = 0
#device = th.device("cuda:"+ str(gpuid))
device = th.device("cpu")

print("Execution device: ",device)
print("PyTorch version: ", th.__version__ )
print("CUDA version: ", th.version.cuda)
print("CUDA device:", th.cuda.get_device_name(gpuid))

Execution device:  cpu
PyTorch version:  1.7.1
CUDA version:  11.0
CUDA device: TITAN V


In [43]:
# Utility functions

# Batched vech2L input V is nb x n(n+1)/2
def bvech2L(V,nb,n):
    count = 0
    L = th.zeros((nb,n,n), device=device, dtype=dtype).clone().detach().requires_grad_(True)
    for j in range(n):
        for i in range(j,n):
            L[...,i,j]=V[...,count]
            count = count + 1
    return L 

# Batched Cholesky decomp
def bcholesky(A):
    L = th.zeros_like(A)

    for i in range(A.shape[-1]):
        for j in range(i+1):
            s = 0.0
            for k in range(j):
                s = s + L[...,i,k].clone() * L[...,j,k].clone()

            L[...,i,j] = th.sqrt(A[...,i,i] - s) if (i == j) else \
                      (1.0 / L[...,j,j].clone() * (A[...,i,j] - s))
    return L

# Batched inverse of lower triangular matrices
def inverseL(L):
    n = L.shape[-1]
    invL = th.zeros_like(L)
    for j in range(0,n):
        invL[...,j,j] = 1.0/L[...,j,j]
        for i in range(j+1,n):
            S = 0.0
            for k in range(i+1):
                S = S - L[...,i,k]*invL[...,k,j].clone()
            invL[...,i,j] = S/L[...,i,i]

    return invL

In [44]:
def test_loop(n=4,m=1000):

    nn = int(n*(n+1)/2)

    th.manual_seed(42)
    X = th.rand((m,nn), device=device, dtype=dtype)
    L = th.add(bvech2L(X,m,n), th.eye(n))
    A = th.matmul(L,th.transpose(L, 1, 2))
    #print("Shape of A {}".format(A.shape))

    start_time = time.time()

    cholA = th.zeros_like(A)
    for i in range(m):
        cholA[i,:,:] = th.cholesky(A[i], upper=False)

    runtime = time.time() - start_time
    print("loop version took {} seconds ".format(runtime))
    return runtime

In [45]:
test_loop(n=10,m=10000)

RuntimeError: bmm does not support automatic differentiation for outputs with complex dtype.

In [30]:
n=3
m=2
nn = int(n*(n+1)/2)
X = th.rand((m,nn))
X
L = th.add( bvech2L(X,m,n), th.eye(n) )
print(L)
A = th.matmul(L,th.transpose(L, 1, 2))
A
cholA = th.zeros_like(A)
cholA
for i in range(m):
    cholA[i,:,:] = th.cholesky(A[i], upper=False)
cholA

tensor([[[1.6802e+00, 0.0000e+00, 0.0000e+00],
         [1.8233e-04, 1.7675e+00, 0.0000e+00],
         [2.5422e-01, 2.4662e-01, 1.6762e+00]],

        [[1.4924e+00, 0.0000e+00, 0.0000e+00],
         [1.5261e-01, 1.5108e+00, 0.0000e+00],
         [3.9220e-01, 7.1507e-01, 1.7660e+00]]], dtype=torch.float64,
       grad_fn=<AddBackward0>)


tensor([[[1.6802e+00, 0.0000e+00, 0.0000e+00],
         [1.8233e-04, 1.7675e+00, 0.0000e+00],
         [2.5422e-01, 2.4662e-01, 1.6762e+00]],

        [[1.4924e+00, 0.0000e+00, 0.0000e+00],
         [1.5261e-01, 1.5108e+00, 0.0000e+00],
         [3.9220e-01, 7.1507e-01, 1.7660e+00]]], dtype=torch.float64,
       grad_fn=<CopySlices>)

In [39]:
def test_batch(n=4,m=1000):

    nn = int(n*(n+1)/2)

    th.manual_seed(42)
    X = th.rand((m,nn), device=device, dtype=dtype)
    L = th.add(bvech2L(X,m,n), th.eye(n))
    A = th.matmul(L,th.transpose(L, 1, 2))
    #print("Shape of A {}".format(A.shape))

    start_time = time.time()

    cholA = th.zeros_like(A)

    cholA = bcholesky(A)

    runtime = time.time() - start_time
    print("batched version took {} seconds ".format(runtime))
    return runtime


In [41]:
test_batch(n=10,m=10000)

batched version took 0.05585980415344238 seconds 


0.05585980415344238